In [25]:
import openai
import os
from dotenv import load_dotenv

# This line loads the variables from your .env file
load_dotenv()

# os.getenv() securely reads the environment variable
api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")

# Fundamental Prompt Engineering


## From Vague to Valuable: The Anatomy of a Great Prompt


### The Bad Prompt


In [5]:
# A bad prompt: vague, unclear, and without direction.
bad_prompt = "Tell me about our IoT device."

response = client.chat.completions.create(
    model="gpt-4o-mini", messages=[{"role": "user", "content": bad_prompt}]
)

# The result will be a generic, unhelpful summary.
print(response.choices[0].message.content)

Sure! To provide you with accurate information about your IoT device, I'll need some details. What kind of IoT device are you referring to? For example, is it a smart home device, a wearable, an industrial sensor, or something else? Additionally, what specific features or functionalities are you interested in?


### The Good Prompt


In [6]:
# A good prompt: specific, contextual, and structured.
good_prompt = """
Analyze the following IoT device log and classify its status.

**Device Log:**
"2024-07-10 14:32:15 - TEMP-007 - Battery: 12% - Signal: Weak - Status: Active"

**Instructions:**
1.  **Task:** Classify the device's overall health as 'Healthy', 'Warning', or 'Critical'.
2.  **Format:** Respond with a JSON object containing 'device_id', 'health_status', and 'reason'.
3.  **Context:** A battery level below 20% is a 'Warning'. A 'Weak' signal is also a 'Warning'.
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={"type": "json_object"},
    messages=[{"role": "user", "content": good_prompt}],
)

# The result will be a precise, structured, and actionable JSON object.
print(response.choices[0].message.content)

{
  "device_id": "TEMP-007",
  "health_status": "Critical",
  "reason": "Battery level is at 12%, which is below 20%, and signal strength is weak."
}


## Pattern 1: Zero-Shot Prompting (Just Ask)


In [7]:
def zero_shot_analysis(log_entry: str) -> str:
    """Analyzes a log entry using a simple zero-shot prompt."""

    prompt = f"""
Analyze this IoT log entry and provide a one-sentence summary of the key issue.

Log Entry: "{log_entry}"

Summary:
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()


log = "2024-07-10 15:00:05 - PUMP-03 - Pressure reading of 250 PSI exceeds maximum threshold of 200 PSI."
print(zero_shot_analysis(log))

The key issue is that PUMP-03 has recorded a pressure reading of 250 PSI, which exceeds the maximum allowable threshold of 200 PSI.


## Pattern 2: Few-Shot Prompting (Show, Don't Just Tell)


In [8]:
def few_shot_classifier(alert_text: str) -> str:
    """Classifies IoT alerts into predefined categories using examples."""

    prompt = f"""
Classify the following IoT alert into one of these categories: [Connectivity, Power, Sensor_Failure, Security].

---
Example 1:
Alert: "Device TEMP-01 failed to check in."
Category: Connectivity

Example 2:
Alert: "Device DOOR-05 battery level at 4%."
Category: Power

Example 3:
Alert: "Device CAM-02 reported anomalous reading outside of normal range."
Category: Sensor_Failure

Example 4:
Alert: "Multiple failed login attempts detected for GATEWAY-01."
Category: Security
---

Now, classify this alert:
Alert: "{alert_text}"
Category:
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,  # Use low temperature for classification tasks
    )
    return response.choices[0].message.content.strip()


print(
    f"Classification: {few_shot_classifier('Sensor HUMID-07 reporting negative humidity values.')}"
)
print(
    f"Classification: {few_shot_classifier('Device LIGHT-10 lost connection to WiFi network.')}"
)

Classification: Category: Sensor_Failure
Classification: Category: Connectivity


## Pattern 3: Chain-of-Thought (CoT) Prompting


In [9]:
def chain_of_thought_diagnosis(symptoms: str) -> str:
    """Diagnoses a complex IoT issue using chain-of-thought reasoning."""

    prompt = f"""
An IoT device is exhibiting the following symptoms:
{symptoms}

Let's think step by step to diagnose the root cause.
1. First, analyze each symptom individually.
2. Then, look for connections or correlations between the symptoms.
3. Based on the connections, formulate the most likely hypotheses for the root cause.
4. Finally, suggest a diagnostic action to confirm the most likely hypothesis.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    return response.choices[0].message.content


symptom_list = """
- Temperature readings are fluctuating wildly.
- Battery level is draining much faster than usual.
- The device disconnects from the network intermittently.
"""

print(chain_of_thought_diagnosis(symptom_list))

Let's break down the symptoms of the IoT device step by step:

### Step 1: Analyze Each Symptom Individually

1. **Temperature Readings Fluctuating Wildly**:
   - Possible causes could include a faulty temperature sensor, environmental interference (e.g., drafts, direct sunlight), or software issues affecting the sensor readings.
   - It could also indicate that the device is experiencing electrical noise or interference.

2. **Battery Level Draining Faster Than Usual**:
   - This could be due to increased power consumption from the device, possibly caused by continuous sensor polling, high-frequency data transmission, or a malfunctioning component.
   - It could also indicate that the device is trying to reconnect to the network frequently, leading to additional power usage.

3. **Device Disconnecting from the Network Intermittently**:
   - This could be caused by weak Wi-Fi signal strength, network congestion, or issues with the device's network interface.
   - It may also be a resul

## Pattern 4: Role-Based Prompting (The Persona Pattern)


In [10]:
def get_expert_analysis(problem_description: str, expert_role: str) -> str:
    """Gets an analysis of a problem from a specific expert's perspective."""

    prompt = f"""
You are a world-class {expert_role}.
Analyze the following situation from your specific professional viewpoint.

Situation: A fleet of 500 IoT delivery drones is experiencing a 15% failure rate in their GPS modules.

Provide your expert analysis, focusing on the key issues, risks, and recommended actions relevant to your role.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


# Get analyses from multiple different experts on the same problem
roles_to_consult = [
    "Hardware Engineer specializing in GPS modules",
    "Network Engineer focused on data transmission",
    "Logistics and Operations Manager",
    "Cybersecurity Analyst concerned with signal spoofing",
]

for role in roles_to_consult:
    print(f"--- Analysis from a {role} ---")
    print(get_expert_analysis("Drone GPS failures", role))
    print("\n")

--- Analysis from a Hardware Engineer specializing in GPS modules ---
**Expert Analysis on GPS Module Failure in IoT Delivery Drones**

### Key Issues

1. **High Failure Rate**:
   - A 15% failure rate in GPS modules is significantly higher than industry standards. Typically, robust GPS modules designed for IoT applications should have failure rates below 5%.

2. **Root Cause Analysis**:
   - There may be several underlying reasons for the high failure rate, including:
     - **Component Quality**: Substandard or incompatible components in the GPS module may lead to failures.
     - **Environmental Factors**: Drones operate in varied and sometimes harsh environments, which may cause wear and tear on GPS components.
     - **Firmware Bugs**: Potential software-related issues could impact the functioning of the GPS modules.
     - **Power Supply Issues**: Inconsistent power supply or battery-related problems could lead to intermittent GPS operation or failures.

3. **Impact on Operations

## A/B Testing Prompts for Optimal Performance


In [12]:
import time


class PromptTester:
    def __init__(self, client):
        self.client = client

    def test_prompt_variants(self, test_input: str, prompt_variants: list[str]) -> dict:
        """Tests multiple prompt variants for a single input."""
        results = {}
        for i, prompt_template in enumerate(prompt_variants):
            variant_name = f"Variant-{i+1}"

            # Fill the template with the test input
            full_prompt = prompt_template.format(input=test_input)

            response = (
                self.client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": full_prompt}],
                )
                .choices[0]
                .message.content
            )

            results[variant_name] = response
            time.sleep(1)  # Simple rate limiting for the test
        return results


# Let's test different ways to ask for a JSON summary of an alert.
alert_text = "Device PUMP-01 reports pressure at 300 PSI. The safe limit is 250 PSI."

variants = [
    # Variant 1: Simple and direct
    "Summarize this alert in JSON: {input}",
    # Variant 2: More specific about keys
    "Extract the device_id, metric, value, and limit from this alert into a JSON object: {input}",
    # Variant 3: Few-shot example
    """
Convert the alert into a structured JSON object.
Example: 'Device TEMP-01 at 50C' -> {{"device_id": "TEMP-01", "metric": "temperature", "value": 50}}
Alert to convert: {input}
""",
]

tester = PromptTester(client=client)
test_results = tester.test_prompt_variants(alert_text, variants)

for variant, result in test_results.items():
    print(f"--- {variant} ---")
    print(result)

--- Variant-1 ---
```json
{
  "device": "PUMP-01",
  "pressure": 300,
  "safe_limit": 250,
  "status": "above safe limit"
}
```
--- Variant-2 ---
Here's the extracted information in a JSON object:

```json
{
  "device_id": "PUMP-01",
  "metric": "pressure",
  "value": 300,
  "limit": 250
}
```
--- Variant-3 ---
Here’s the alert converted into a structured JSON object:

```json
{
  "device_id": "PUMP-01",
  "metric": "pressure",
  "value": 300,
  "safe_limit": 250
}
```


## Putting It All Together: A Smart IoT Diagnostic System


In [15]:
import json
from datetime import datetime
from typing import Dict


class IoTDiagnosticSystem:
    def __init__(self, client):
        self.client = client

    def generate_report(self, alert_log: str, device_context: Dict) -> Dict:
        """Generates a complete diagnostic report for an IoT alert."""

        # 1. Classify the alert using a Few-Shot Prompt
        classification_prompt = f"""
Classify this alert log into one of [Power, Connectivity, Sensor_Failure, Software_Bug].
Example 1: "Battery at 3%" -> Power
Example 2: "Failed to connect to WiFi" -> Connectivity
Log: "{alert_log}"
Category:
"""
        category = (
            self.client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=0,
                messages=[{"role": "user", "content": classification_prompt}],
            )
            .choices[0]
            .message.content
        )
        print("[DEBUG] Category: ", category)

        # 2. Get a detailed diagnosis using Chain-of-Thought
        diagnosis_prompt = f"""
An IoT device reported the following alert: "{alert_log}".
Device context: {device_context}

Let's diagnose this step by step:
1.  What is the immediate problem indicated by the alert?
2.  Considering the device context, what are the 2-3 most likely root causes?
3.  What is the recommended first action to take to verify the cause?
"""
        diagnosis = (
            self.client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=0.3,
                messages=[{"role": "user", "content": diagnosis_prompt}],
            )
            .choices[0]
            .message.content
        )
        print("[DEBUG] Diagnosis: ", diagnosis)

        # 3. Get a specialized opinion using Role-Based Prompting
        security_prompt = f"""
You are a cybersecurity expert. Analyze this IoT alert for potential security implications:
Alert: "{alert_log}"
Context: {device_context}
"""
        security_analysis = (
            self.client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=0.2,
                messages=[{"role": "user", "content": security_prompt}],
            )
            .choices[0]
            .message.content
        )
        print("[DEBUG] Security Analysis: ", security_analysis)

        # 4. Assemble the final report
        report = {
            "alert": alert_log,
            "category": category,
            "diagnosis_reasoning": diagnosis,
            "security_assessment": security_analysis,
            "report_generated_at": datetime.now().isoformat(),
        }
        return report


# --- Demo of the complete system ---
system = IoTDiagnosticSystem(client=client)

test_alert = "Device MOTION-07 is spamming the server with thousands of 'motion detected' events per second."
test_context = {
    "device_type": "PIR Motion Sensor",
    "firmware_version": "v1.2.1",
    "last_firmware_update": "2024-06-01",
    "location": "Main entrance",
}

full_report = system.generate_report(test_alert, test_context)

print(json.dumps(full_report, indent=2))

[DEBUG] Category:  Software_Bug
[DEBUG] Diagnosis:  ### 1. Immediate Problem Indicated by the Alert
The immediate problem indicated by the alert is that the PIR Motion Sensor (Device MOTION-07) is generating an excessive number of 'motion detected' events, spamming the server with thousands of alerts per second. This can lead to server overload, data loss, and can hinder the performance of the overall IoT system.

### 2. Most Likely Root Causes
Considering the device context, the 2-3 most likely root causes for this issue could be:

1. **Environmental Factors**: The sensor may be detecting motion due to environmental factors such as wind, animals, or other non-human movements. If the sensor is overly sensitive or improperly installed, it may trigger false positives.

2. **Firmware Bug**: There could be a bug in the current firmware version (v1.2.1) that causes the device to malfunction and send repeated alerts. This could be a known issue that has been addressed in a later firmware ver

# Advanced Prompting Strategies


In [17]:
complex_problem = """
A smart factory's assembly line has stopped. Key symptoms include:
- Conveyor belt motors in Zone C are offline.
- Vibration sensors in the adjacent Zone B are showing erratic, high-amplitude readings.
- Network latency for all devices in Zone C has increased by 500ms.
- The issue began an hour after the morning shift started.
- No new firmware has been deployed recently.
"""

## Strategy 1: The Self-Consistency Method


In [16]:
import json
from collections import Counter
from typing import List, Dict, Any


def self_consistency_diagnosis(problem: str, num_runs: int = 5) -> str:
    """Runs a diagnosis prompt multiple times to find the most consistent root cause."""

    prompt = f"""
Analyze the following IoT issue and state the single most likely root cause.

Problem: {problem}

Most Likely Root Cause:
"""

    responses = []
    print(f"Running diagnosis {num_runs} times to find consensus...")

    for i in range(num_runs):
        response = (
            client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,  # Higher temperature encourages diverse reasoning paths
            )
            .choices[0]
            .message.content
        )

        # We only care about the stated root cause for this simple example
        root_cause = response.strip()
        print(f"  Run {i+1} Diagnosis: {root_cause}")
        responses.append(root_cause)

    # Find the most common response using a Counter
    if not responses:
        return "Failed to get any valid responses."

    most_common_cause, count = Counter(responses).most_common(1)[0]

    return f"Consensus Diagnosis (agreed upon in {count}/{num_runs} runs): {most_common_cause}"


# A problem where a single prompt might give different answers
device_issue = "A smart thermostat is reading 10°F higher than the actual room temperature. It was installed last week and has the latest firmware."

print(self_consistency_diagnosis(device_issue))

Running diagnosis 5 times to find consensus...
  Run 1 Diagnosis: The most likely root cause of the smart thermostat reading 10°F higher than the actual room temperature is an incorrect installation or placement of the thermostat. If the thermostat is installed near a heat source (such as a vent, oven, or direct sunlight), it can cause it to register a higher temperature than the surrounding air. Additionally, if it is improperly calibrated or has inadequate airflow around it, that could also lead to inaccurate readings.
  Run 2 Diagnosis: The single most likely root cause of the smart thermostat reading 10°F higher than the actual room temperature is improper placement or installation of the thermostat. If the thermostat is located near a heat source (such as a radiator, vent, or direct sunlight) or in a poorly ventilated area, it may inaccurately sense the temperature, leading to an inflated reading.
  Run 3 Diagnosis: The single most likely root cause of the smart thermostat reading

## Strategy 2: Tree-of-Thought (ToT) Prompting


In [19]:
class TreeOfThoughtSolver:
    def __init__(self, client):
        self.client = client

    def solve(self, problem: str, num_paths: int = 3) -> str:
        # 1. Generate initial thoughts (the "branches" of the tree)
        print("--- Stage 1: Generating initial reasoning paths ---")
        initial_thoughts = self._generate_initial_thoughts(problem, num_paths)
        print("Initial Paths:", initial_thoughts)

        # 2. Develop each thought into a more detailed plan
        print("\n--- Stage 2: Developing each path ---")
        developed_paths = []
        for thought in initial_thoughts:
            developed_paths.append(self._develop_path(problem, thought))

        # 3. Evaluate the developed paths to find the best one
        print("\n--- Stage 3: Evaluating paths and synthesizing a final answer ---")
        return self._evaluate_and_synthesize(problem, developed_paths)

    def _generate_initial_thoughts(self, problem: str, num_thoughts: int) -> list[str]:
        prompt = f"""
Problem: "{problem}"
Generate {num_thoughts} distinct, high-level approaches to diagnose this problem. Each approach should be a single sentence.
"""
        response = (
            self.client.chat.completions.create(
                model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
            )
            .choices[0]
            .message.content
        )
        return [line.strip() for line in response.split("\n") if line.strip()]

    def _develop_path(self, problem: str, thought: str) -> str:
        prompt = f"""
Problem: "{problem}"
Reasoning Path: "{thought}"
Elaborate on this reasoning path. What are the specific steps you would take to investigate this? What are the pros and cons of this approach?
"""
        return (
            self.client.chat.completions.create(
                model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
            )
            .choices[0]
            .message.content
        )

    def _evaluate_and_synthesize(self, problem: str, developed_paths: list[str]) -> str:
        paths_text = "\n\n".join(
            f"Path {i+1}:\n{path}" for i, path in enumerate(developed_paths)
        )
        prompt = f"""
You have explored several reasoning paths to solve this problem: "{problem}"

Here are the explored paths:
{paths_text}

Now, act as an expert reviewer. Evaluate the paths and synthesize the best elements into a single, optimal action plan.
"""
        return (
            self.client.chat.completions.create(
                model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}]
            )
            .choices[0]
            .message.content
        )


# Let's use ToT to solve our complex factory problem
tot_solver = TreeOfThoughtSolver(client=client)
solution = tot_solver.solve(complex_problem)
print("\n--- Final ToT Solution ---")
print(solution)

--- Stage 1: Generating initial reasoning paths ---
Initial Paths: ["1. Conduct a comprehensive network diagnostic to identify potential connectivity issues or interference affecting Zone C's devices and analyze if the increased latency correlates with any specific changes in Zone B's sensor readings.", '2. Perform a thorough equipment inspection in Zone C, focusing on the conveyor belt motors and associated power supply, while simultaneously reviewing historical logs and sensor data from Zone B to correlate any mechanical failures.', '3. Initiate a cross-disciplinary team meeting that includes IT, engineering, and operations to collaboratively analyze the timeline of the incident, discuss potential internal changes or external factors that could have influenced the erratic behavior of the systems involved.']

--- Stage 2: Developing each path ---

--- Stage 3: Evaluating paths and synthesizing a final answer ---

--- Final ToT Solution ---
### Optimal Action Plan for Investigating the

## Strategy 3: ReAct (Reasoning and Acting)


In [22]:
class ReActAgent:
    def __init__(self, available_tools: dict, client):
        self.client = client
        self.tools = available_tools

    def run(self, problem: str, max_steps: int = 5):
        thought_history = ""
        for i in range(max_steps):
            print(f"\n--- ReAct Step {i+1} ---")

            # 1. Reason about the next action
            prompt = f"""
You are an IoT diagnostic agent. Solve the problem by reasoning and then taking an action.
Problem: "{problem}"
Thought History:
{thought_history}

Your available tools are: {list(self.tools.keys())}

Thought: Analyze the current situation and decide on the next best action to take.
Action: [Choose one tool from the list]
"""
            response = (
                self.client.chat.completions.create(
                    model="gpt-4o-mini",
                    temperature=0,
                    messages=[{"role": "user", "content": prompt}],
                )
                .choices[0]
                .message.content
            )

            # Parse the thought and action
            thought = response.split("Action:")[0].replace("Thought:", "").strip()
            action = response.split("Action:")[1].strip()
            print(f"Thought: {thought}")
            print(f"Action: {action}")

            # 2. Execute the chosen action
            if action in self.tools:
                tool_result = self.tools[
                    action
                ]()  # In a real app, you'd pass arguments
                print(f"Observation: {tool_result}")

                # Update the history for the next loop
                thought_history += f"Step {i+1}:\nThought: {thought}\nAction: {action}\nObservation: {tool_result}\n"

                # A simple check for a final answer
                if "root cause is" in tool_result.lower():
                    print("\n--- Diagnosis Complete ---")
                    return tool_result
            else:
                print("Action not recognized. Ending loop.")
                break

        return "Could not determine root cause within the step limit."


# --- Define some dummy tools for the demo ---
def check_network_logs():
    return "Network logs show high latency in Zone C, starting at 09:05 AM."


def check_motor_status():
    return "All motors in Zone C are reporting 'offline' via the API."


def review_maintenance_schedule():
    return "No recent maintenance was scheduled for Zone C. However, a power grid switchover occurred at 09:02 AM."


# --- Run the ReAct agent ---
tools = {
    "check_network_logs": check_network_logs,
    "check_motor_status": check_motor_status,
    "review_maintenance_schedule": review_maintenance_schedule,
}

react_agent = ReActAgent(tools, client)
react_agent.run(complex_problem)


--- ReAct Step 1 ---
Thought: Given the symptoms presented, it seems that the issue may be related to network connectivity or motor functionality in Zone C, especially since the conveyor belt motors are offline and there is increased network latency. The erratic readings from the vibration sensors in Zone B could indicate a problem that is affecting both zones.

Since the network latency has increased significantly, it would be prudent to first check the network logs to identify any potential issues that could be causing the latency and affecting the devices in Zone C.
Action: check_network_logs
Observation: Network logs show high latency in Zone C, starting at 09:05 AM.

--- ReAct Step 2 ---
Thought: Given the current situation, the high latency in Zone C starting at 09:05 AM suggests a potential network issue that could be affecting the functionality of the conveyor belt motors. Since the motors are offline, it is crucial to determine if they are functioning properly or if the issue

'Could not determine root cause within the step limit.'

## Strategy 4: Multi-Agent Debate


In [24]:
class MultiAgentDebate:
    def __init__(self, client):
        self.client = client

    def create_agents(self, problem: str) -> dict:
        """Create agents with different perspectives and expertise."""

        agents = {
            "optimist": {
                "role": "Optimistic Analyst",
                "perspective": "Focus on opportunities and positive outcomes. Consider best-case scenarios and potential benefits.",
                "expertise": "Risk assessment and opportunity identification",
            },
            "pessimist": {
                "role": "Conservative Analyst",
                "perspective": "Focus on risks and potential problems. Consider worst-case scenarios and potential downsides.",
                "expertise": "Risk mitigation and contingency planning",
            },
            "pragmatist": {
                "role": "Practical Engineer",
                "perspective": "Focus on practical implementation and feasibility. Consider resource constraints and real-world limitations.",
                "expertise": "Implementation feasibility and resource optimization",
            },
        }

        return agents

    def run_debate(self, problem: str, rounds: int = 3) -> str:
        """Run a structured debate between multiple agents."""

        agents = self.create_agents(problem)
        debate_history = []

        print("=== Multi-Agent Debate Session ===")
        print(f"Problem: {problem}\n")

        # Initial analysis from each agent
        initial_views = {}
        for agent_id, agent_info in agents.items():
            prompt = f"""
You are a {agent_info['role']} with expertise in {agent_info['expertise']}.
Your perspective: {agent_info['perspective']}

Problem: {problem}

Provide your initial analysis and recommendations based on your perspective.
"""
            response = (
                self.client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7,
                )
                .choices[0]
                .message.content
            )

            initial_views[agent_id] = response
            print(f"--- {agent_info['role']} Initial View ---")
            print(response)
            print()

        # Debate rounds
        for round_num in range(rounds):
            print(f"=== Debate Round {round_num + 1} ===")

            for agent_id, agent_info in agents.items():
                # Create context from other agents' views
                other_views = {k: v for k, v in initial_views.items() if k != agent_id}
                other_views_text = "\n\n".join(
                    [f"{agents[k]['role']}: {v}" for k, v in other_views.items()]
                )

                prompt = f"""
You are a {agent_info['role']} with expertise in {agent_info['expertise']}.
Your perspective: {agent_info['perspective']}

Problem: {problem}

Other agents' views:
{other_views_text}

Your previous view:
{initial_views[agent_id]}

Based on the other agents' perspectives, provide your updated analysis. 
Consider their points, address any concerns, and refine your recommendations.
"""

                response = (
                    self.client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=[{"role": "user", "content": prompt}],
                        temperature=0.7,
                    )
                    .choices[0]
                    .message.content
                )

                initial_views[agent_id] = response
                print(f"--- {agent_info['role']} Updated View ---")
                print(response)
                print()

        # Final synthesis
        all_views = "\n\n".join(
            [f"{agents[k]['role']}: {v}" for k, v in initial_views.items()]
        )

        synthesis_prompt = f"""
Problem: {problem}

After a structured debate, here are the final positions of three expert agents:

{all_views}

Synthesize these perspectives into a comprehensive, balanced solution that addresses:
1. The opportunities and benefits identified
2. The risks and concerns raised  
3. The practical implementation considerations

Provide a final recommendation that incorporates the best elements from all perspectives.
"""

        final_synthesis = (
            self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": synthesis_prompt}],
                temperature=0.3,
            )
            .choices[0]
            .message.content
        )

        return final_synthesis


# Example usage
debate_system = MultiAgentDebate(client=client)
complex_decision = "Should we implement a new AI-powered predictive maintenance system in our factory? Consider costs, benefits, risks, and implementation challenges."

debate_result = debate_system.run_debate(complex_decision)
print("=== Final Synthesis ===")
print(debate_result)

=== Multi-Agent Debate Session ===
Problem: Should we implement a new AI-powered predictive maintenance system in our factory? Consider costs, benefits, risks, and implementation challenges.

--- Optimistic Analyst Initial View ---
### Initial Analysis and Recommendations for Implementing an AI-Powered Predictive Maintenance System

#### Overview
The implementation of an AI-powered predictive maintenance system in your factory presents a significant opportunity to enhance operational efficiency, reduce costs, and improve overall productivity. By leveraging predictive analytics, your organization can proactively address maintenance needs, minimizing downtime and maximizing asset utilization.

#### Cost-Benefit Analysis

##### Costs:
- **Initial Investment**: There will be upfront costs associated with purchasing the software, hardware, and potential upgrades to existing machinery.
- **Training and Integration**: Employees will require training to effectively use the new system, and ther

## Strategy 5: Reflexion


In [27]:
class ReflexionAgent:
    def __init__(self, client):
        self.client = client
        self.reflection_history = []

    def solve_with_reflection(self, problem: str, max_attempts: int = 3) -> str:
        """Solve a problem with iterative reflection and improvement."""

        print(f"=== Reflexion Agent Solving: {problem} ===\n")

        for attempt in range(max_attempts):
            print(f"--- Attempt {attempt + 1} ---")

            # Generate solution
            solution = self._generate_solution(problem, attempt)
            print(f"Solution: {solution}")

            # Reflect on the solution
            reflection = self._reflect_on_solution(problem, solution, attempt)
            print(f"Reflection: {reflection}")

            # Check if we should continue or if solution is satisfactory
            if self._is_solution_satisfactory(reflection):
                print(f"\n=== Solution Found on Attempt {attempt + 1} ===")
                return solution

            # Store reflection for next attempt
            self.reflection_history.append(
                {"attempt": attempt + 1, "solution": solution, "reflection": reflection}
            )

            print()

        # If we reach here, return the best solution from all attempts
        return self._synthesize_best_solution(problem)

    def _generate_solution(self, problem: str, attempt: int) -> str:
        """Generate a solution based on the problem and previous reflections."""

        reflection_context = ""
        if self.reflection_history:
            reflection_context = "\n\nPrevious attempts and reflections:\n"
            for ref in self.reflection_history:
                reflection_context += f"Attempt {ref['attempt']}: {ref['reflection']}\n"

        prompt = f"""
Problem: {problem}

{reflection_context}

Based on the problem and any previous reflections, provide a solution.
If this is not the first attempt, incorporate lessons learned from previous attempts.
"""

        response = (
            self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
            )
            .choices[0]
            .message.content
        )

        return response

    def _reflect_on_solution(self, problem: str, solution: str, attempt: int) -> str:
        """Reflect on the quality and completeness of the solution."""

        prompt = f"""
Problem: {problem}
Solution Attempt {attempt + 1}: {solution}

Critically evaluate this solution:
1. Does it fully address the problem?
2. Are there any logical flaws or gaps?
3. Could it be improved or refined?
4. What specific improvements would you suggest?

Provide a detailed reflection that will help improve the next attempt.
"""

        response = (
            self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
            )
            .choices[0]
            .message.content
        )

        return response

    def _is_solution_satisfactory(self, reflection: str) -> bool:
        """Determine if the solution is satisfactory based on reflection."""

        # Simple heuristic: if reflection doesn't mention major issues, consider it satisfactory
        negative_indicators = [
            "major flaw",
            "significant gap",
            "doesn't address",
            "incomplete",
            "missing",
        ]
        return not any(
            indicator in reflection.lower() for indicator in negative_indicators
        )

    def _synthesize_best_solution(self, problem: str) -> str:
        """Synthesize the best elements from all attempts."""

        attempts_summary = "\n\n".join(
            [
                f"Attempt {ref['attempt']}:\nSolution: {ref['solution']}\nReflection: {ref['reflection']}"
                for ref in self.reflection_history
            ]
        )

        prompt = f"""
Problem: {problem}

After multiple attempts, here are the solutions and reflections:

{attempts_summary}

Synthesize the best elements from all attempts into a comprehensive final solution.
Address the issues identified in the reflections and combine the strongest aspects of each attempt.
"""

        response = (
            self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
            )
            .choices[0]
            .message.content
        )

        return response


# Example usage
reflexion_agent = ReflexionAgent(client=client)
complex_problem = "Design a comprehensive IoT security strategy for a smart city that balances functionality, privacy, and cost-effectiveness."

final_solution = reflexion_agent.solve_with_reflection(complex_problem)
print("\n=== Final Reflexion Solution ===")
print(final_solution)

=== Reflexion Agent Solving: Design a comprehensive IoT security strategy for a smart city that balances functionality, privacy, and cost-effectiveness. ===

--- Attempt 1 ---
Solution: Designing a comprehensive IoT security strategy for a smart city requires a multifaceted approach that balances functionality, privacy, and cost-effectiveness. Below is a structured solution that incorporates lessons learned from previous strategies and best practices in the field.

### 1. **Assessment of Current Infrastructure**
   - **Inventory of IoT Devices**: Conduct a thorough inventory of all IoT devices in use, including sensors, cameras, traffic lights, and other smart systems.
   - **Vulnerability Assessment**: Evaluate the security posture of existing devices, including firmware versions, manufacturer support, and known vulnerabilities.

### 2. **Establishing a Security Governance Framework**
   - **Policy Development**: Create comprehensive security policies that outline roles, responsibilit